# Using the Staff Training Scheduler Tool

The following tutorial is a step-by-step guide to using the staff query and training scheduler tool in the native python. When we get to the actual scheduling tool, I'll briefly explain the mathematical assumptions underlying the tool, though for those of you who are only interested in using the app I've built, feel free to skip said section. 

Let's begin. The first thing that we'll need is to generate a training matrix for all the staff in the company. Since this program was written originally for BrainStorm STEM Edu LLC (BrainStorm), I'll be referring to the documents they provided for the completion of this project. BrainStorm in particular had two different concurrent programs that needed to be trained for, so two different sets of documents are included here. I've included in this tutorial how to switch between documents.

### Creating training matrices and the training needs doc

##### Training matrices
This app works by creating a set of training matrices that coincide with the training needs of your staff. To do this, we leverage training_matrices.py.

We'll need to start by importing the package and starting a query session.

In [ ]:
from query.training_schedule.training_matrices import *
training_needs = query(staff_skillsets='input-data/skills.csv', 
                       courses='input-data/staff-1w.csv', 
                       required_skills='input-data/class_by_topic_1w.csv')

This will automatically initialize a set of training matrices, generated from the number of topics in the skills.csv doc, with as many empty 1-hot matrices as there are staff members. As an example, consider the following:

Let's say we have three staff members--'A', 'B', and 'C'--and our organization offers after school courses in the following three areas--'Python', 'Lego', and 'Volcanoes'. The initial training matrices for this should look like the following:
    
    [0, 0, 0]
    [0, 0, 0]
    [0, 0, 0]

And we can think of this matrix like a simple table

       'Python'    'Lego'    'Volcanoes'
    A    0           0           0
    B    0           0           0
    C    0           0           0

Next, and without any additional intervention, the query tool will find the names of every staff member in the courses worksheet, and for every course they need to teach, will find what skills they NEED to have in the required_skills worksheet. It then checks the staff_skillsets worksheet if they need training on that skill, and if they do, it replaces the zero corresponding with that skill to a 1 in their training matrix. So let's say 'A' needs training on 'Volcanoes'. We update the training matrices to look like this:

    [0, 0, 1]
    [0, 0, 0]
    [0, 0, 0]

or, visually again, like this:

      'Python'    'Lego'    'Volcanoes'
    A    0           0           1
    B    0           0           0
    C    0           0           0

We do this for every staff member, so if 'C' needs training in 'Python' and 'Lego', but not 'Volcanoes', their matrix should look like this:

      'Python'    'Lego'    'Volcanoes'
    A    0           0           1
    B    0           0           0
    C    1           1           0

BrainStorm however has a couple of programs running concurrently over the summer, both of which will require training for staff. To ADD data to your training matrices from another course, we can swap out our datasheets and add new data to our training matrices using the swap_courses function.

In [ ]:
training_needs.swap_courses('input-data/staff-3w.csv', 'input-data/class_by_topic_3w.csv')

Sometimes, we may need to update who needs training with more people. While this is probably pretty rare, you can swap that document out using the add_more_staff function.

In [ ]:
training_needs.add_more_staff('NEW/STAFF/SKILLS/WORKSHEET')

Once we've created these matrices, these in and of themselves are incredibly useful. So let's save them to a file for later.

In [ ]:
df_tr_needs = training_needs.matrices_to_csv('PATH/TO/FILE')

This will thus save our data to some arbitrary point in our computer to use later. Additionally, this step returns a pandas dataframe (which is more or less the Python version of an SQL table) which we'll use in the next step--creating a training schedule.


### Creating a training schedule
We now have a query-able document containing the training needs of our staff members. Let's use that to build a schedule. To do that, we'll use the scheduler package and initializing a scheduler instance.

In [ ]:
from query.training_schedule.scheduler import *
scheduling = scheduler()

The scheduler works by taking in a document organized in rows representing your staff and columns representing topics people need to be trained on (In other words, the formatting for the training matrices), and finding courses that are semantically dissimilar. I define semantic similarity in this instance as having similar people who need to take the course--in other words, are there overlapping individuals who need this training? To find that, we can use cossine similarity. I'll explain how this works below, but let's generate our data first.

In our example below, we'll define some information about our courses a priori. First, do we want to take the time to set up training for courses that have only 1 or 2 people who need training? Probably not. So for the min_participants parameter, we set it to 3 in order to indicate that we only want courses that have three or more people who need to learn it. We also pass the function the dataframe for our training matrices.

In [ ]:
scheduling.cosine_concurrency_doc(df_tr_needs, min_participants=3)

Now, this will generate a series of new matrices representing whether or not there is a conflict in the people who need to take the class. It stores this data in a table like so:
            
            topic1 .  topic 2 .  topic3
    topic 1   0         0          0
    topic 2   0         0          0
    topic 3   0         0          0

If there is no conflict between two courses, it changes the zero to a 1 so we can find training courses that can be run concurrently easier:

            
            topic1 .  topic 2 .  topic3
    topic 1   0         1          0
    topic 2   1         0          0
    topic 3   0         0          0


Now, we need to take this table and turn it into a schedule! That last step is quite simple: we can use the create_schedule function, which will take the data above as inputs and output a data frame of what courses can be run concurrently. This function needs to know as a parameter how many rooms you have available however for training. Below, I set it to six, though one can do more or less as needed.

In [ ]:
df_schedule = scheduling.create_schedule(n_rooms=6)

One should note that this algorithm is "greedy". It will attempt to group as many classes together as it can in order to reduce the number of slots in the day required for training.

And there you have it! A functional training schedule!